# Weather Lookup
Dieses Notebook zeigt, wie Wetterdaten für Tripdaten anhand von Breite, Länge und Startzeit abgefragt werden können.

In [ ]:
import pandas as pd
import requests
from datetime import datetime
import sys
from pathlib import Path
sys.path.append(str(Path('src').resolve()))
from db import open_connection


In [ ]:
conn = open_connection()
query = "SELECT trip_id, start_lat, start_lon, start_time FROM public.trips LIMIT 5;"
trips = pd.read_sql_query(query, conn)
conn.close()
trips

In [ ]:
def fetch_temperature(lat, lon, timestamp):
    date = pd.to_datetime(timestamp).date()
    url = ("https://api.open-meteo.com/v1/forecast?"
           f'latitude={lat}&longitude={lon}&hourly=temperature_2m'
           f'&start_date={date}&end_date={date}')
    resp = requests.get(url, timeout=10)
    resp.raise_for_status()
    data = resp.json()
    times = data['hourly']['time']
    temps = data['hourly']['temperature_2m']
    df = pd.DataFrame({'time': times, 'temp': temps})
    df['time'] = pd.to_datetime(df['time'])
    row = df.loc[df['time'] == pd.to_datetime(timestamp)]
    if not row.empty:
        return float(row['temp'].iloc[0])
    return None


In [ ]:
trips['temperature'] = trips.apply(lambda r: fetch_temperature(r.start_lat, r.start_lon, r.start_time), axis=1)
trips